In [81]:
# Imports necesarios
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold
from tqdm import tqdm  # Para mostrar el progreso
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,Input, Concatenate
from keras.models import Model
#from keras.layers import  Dense, Dropout


In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
# Cargar el archivo .npz
loaded_data = np.load('/content/drive/MyDrive/DeepLearning/data_arrays.npz')

# Extraer los arrays
X_train_np = loaded_data['X_train']
X_val_np = loaded_data['X_val']
X_test_np = loaded_data['X_test']
y_train_np = loaded_data['y_train']
y_val_np = loaded_data['y_val']
y_test_np = loaded_data['y_test']

In [84]:
# Cargar el archivo .npz
loaded_image_data = np.load('/content/drive/MyDrive/DeepLearning/preprocessed_images.npz')

# Extraer los arrays de imágenes preprocesadas
X_train_preprocessed = loaded_image_data['X_train_preprocessed']
X_val_preprocessed = loaded_image_data['X_val_preprocessed']
X_test_preprocessed = loaded_image_data['X_test_preprocessed']

In [85]:
# Cargar los modelos
model_tabular = load_model('/content/drive/MyDrive/DeepLearning/modelo_tabular.keras')
model_images = load_model('/content/drive/MyDrive/DeepLearning/modelo_resnet50.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [86]:
# Asegurarse de que los modelos no sean entrenables
model_tabular.trainable = False
model_images.trainable = False

In [7]:
model_tabular.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                     │ (None, 64)                  │           1,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 3)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,776 (34.29 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,387 (17.14 KB)

 Optimizer params: 4,389 (17.15 KB)

In [8]:
model_images.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,950,088 (98.99 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 24,768,899 (94.49 MB)

 Optimizer params: 1,181,189 (4.51 MB)

In [89]:
# Extraer características del modelo tabular
features_tabular_train = model_tabular.predict(X_train_np)
features_tabular_val = model_tabular.predict(X_val_np)
features_tabular_test = model_tabular.predict(X_test_np)

# Extraer características del modelo de imágenes
features_images_train = model_images.predict(X_train_preprocessed)
features_images_val = model_images.predict(X_val_preprocessed)
features_images_test = model_images.predict(X_test_preprocessed)


59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 113s 6s/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 117s 6s/step


In [90]:
# Concatenar las características
X_train_fused = np.concatenate((features_tabular_train, features_images_train), axis=1)
X_val_fused = np.concatenate((features_tabular_val, features_images_val), axis=1)
X_test_fused = np.concatenate((features_tabular_test, features_images_test), axis=1)

In [91]:
# Convertir etiquetas a formato categórico (one-hot encoding)
y_train_categorical = to_categorical(y_train_np, num_classes=3)
y_val_categorical = to_categorical(y_val_np, num_classes=3)
y_test_categorical = to_categorical(y_test_np, num_classes=3)

## Fusion 1

In [93]:
#  Definir el modelo fusionado con Dropout
model_fused = Sequential()
model_fused.add(Dense(128, activation='relu', input_shape=(X_train_fused.shape[1],)))
model_fused.add(Dropout(0.5))  # Regularización con Dropout
model_fused.add(Dense(64, activation='relu'))
model_fused.add(Dropout(0.5))  # Regularización con Dropout
model_fused.add(Dense(3, activation='softmax'))  # Tres clases para clasificación multiclase

#  Compilar el modelo
model_fused.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#  Definir early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#  Entrenar el modelo usando el conjunto de validación
model_fused.fit(X_train_fused, y_train_categorical,
                epochs=10,
                batch_size=32,
                validation_data=(X_val_fused, y_val_categorical),  # Usa tu conjunto de validación
                callbacks=[early_stopping])
loss, accuracy = model_fused.evaluate(X_test_fused, y_test_categorical)
print(f'Loss: {loss}, Accuracy: {accuracy}')
model_fused.summary()


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5906 - loss: 0.9297 - val_accuracy: 0.6262 - val_loss: 0.8002
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8987 - loss: 0.3701 - val_accuracy: 0.6150 - val_loss: 1.0102
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9119 - loss: 0.2681 - val_accuracy: 0.6102 - val_loss: 1.1589
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9033 - loss: 0.2651 - val_accuracy: 0.6150 - val_loss: 1.1681
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6640 - loss: 0.7735 
Loss: 0.8030547499656677, Accuracy: 0.6411483287811279


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_55 (Dense)                     │ (None, 128)                 │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,043 (109.55 KB)

 Trainable params: 9,347 (36.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,696 (73.04 KB)

## Fusion  2

In [80]:
# Modificar ambos modelos para obtener la penúltima capa

# Para el modelo tabular, crear un nuevo modelo basado en la arquitectura original
input_tabular = Input(shape=(X_train_np.shape[1],))  # Definir la entrada para datos tabulares
output_tabular = model_tabular(input_tabular)  # Llamar al modelo tabular con la nueva entrada
model_tabular_modified = Model(inputs=input_tabular, outputs=model_tabular.layers[-2].output)  # Penúltima capa
# Modificar el modelo de imágenes para obtener la penúltima capa
model_images_modified = Model(inputs=model_images.input, outputs=model_images.layers[-2].output)


# Definir las entradas para el modelo fusionado
input_images = Input(shape=(224, 224, 3))  # Entrada para el modelo de imágenes
input_tabular = Input(shape=(X_train_np.shape[1],))  # Entrada para datos tabulares

# Obtener características directamente desde los modelos modificados
features_images = model_images_modified(input_images)
features_tabular = model_tabular_modified(input_tabular)

# Comprimir características de imágenes a la misma dimensión que las características tabulares
compressed_images = Dense(features_tabular.shape[1], activation='relu')(features_images)

# Fusionar características
fused_features = Concatenate()([compressed_images, features_tabular])

# Construir el clasificador final
x = Dense(256, activation='relu')(fused_features)
x = Dropout(0.5)(x)  # Regularización con Dropout
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularización con Dropout
output = Dense(3, activation='softmax')(x)  # Asumimos que hay 3 clases

# Crear el modelo final
model_fused1 = Model(inputs=[input_images, input_tabular], outputs=output)

# Compilar el modelo
model_fused1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Imagenes",features_images.shape)
print("Imagenes comprimida", compressed_images.shape)
print("Tabular", features_tabular.shape)
model_fused1.summary()

Imagenes (None, 256)
Imagenes comprimida (None, 16)
Tabular (None, 16)


Model: "functional_60"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_47            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_59             │ (None, 256)            │     24,768,128 │ input_layer_47[0][0]   │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_48            │ (None, 26)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_48 (Dense)          │ (None, 16)             │          4,112 │ functional_59[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_58             │ (None, 16)             │          4,336 │ input_layer_48[0][0]   │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_12            │ (None, 32)             │              0 │ dense_48[0][0],        │
│ (Concatenate)             │                        │                │ functional_58[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_49 (Dense)          │ (None, 256)            │          8,448 │ concatenate_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_24 (Dropout)      │ (None, 256)            │              0 │ dense_49[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_50 (Dense)          │ (None, 128)            │         32,896 │ dropout_24[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 128)            │              0 │ dense_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_51 (Dense)          │ (None, 3)              │            387 │ dropout_25[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 24,818,307 (94.67 MB)

 Trainable params: 45,843 (179.07 KB)

 Non-trainable params: 24,772,464 (94.50 MB)

In [73]:
# Entrenar el modelo fusionado
model_fused.fit([X_train_preprocessed, X_train_np], y_train_categorical,
                 epochs=10,
                 batch_size=32,
                 validation_data=([X_val_preprocessed, X_val_np], y_val_categorical))



In [74]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model_fused.evaluate([X_test_preprocessed, X_test_np], to_categorical(y_test_np, num_classes=3))
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')